## Aufgabe 1: Vorhersage Absatzzahlen

In [1]:
%matplotlib inline
import psycopg2
import pandas as pd
import numpy as np
from fbprophet import Prophet

In [2]:
%load_ext autoreload
%autoreload 2 

In [3]:
conn = psycopg2.connect("dbname=eveonline user=REPLACE_USER")
cur = conn.cursor()

In [36]:
def demand_for_product_id(pid):
    cur.execute("SELECT time as ds, sum(sales_units) as y FROM items WHERE typeid={0} GROUP BY time ORDER BY time"
               .format(pid))
    data = cur.fetchall()
    df = pd.DataFrame(data, columns=['ds', 'y'])
    df['y'] = np.log(df['y'])

    return df

In [37]:
products = {
    'tengu': 29984,
    'mackinaw': 22548,
    'ishtar': 22548,
    'hulk': 22544,
    'dominix': 645,
    'caldari_controllsys': 15681,
    'covert_ops': 11578,
    'navi_stasis_covert': 17559,
    'adaptive_inv_field': 2281,
    'rep_field_gyrostab': 15806,
    'nanite': 28668,
    'sisters_cs_probe': 30488,
    'navy_cap_booster_800': 32014,
    'scourge_fmissile': 2629,
    'navy_cap_booster_400': 32006,
    'tritanium': 34,
    'mexallon': 36,
    'pyerite': 35,
    'nocxium': 38,
    'isogen': 37
}

In [ ]:
df_caldari_controllsys = demand_for_product()
df_covert_ops = demand_for_product()
df_navi_stasis_covert = demand_for_product()
df_adaptive_inv_field = demand_for_product()
df_rep_field_gyrostab = demand_for_product()

---
## Imputation

In [38]:
"""
Creates a full list of dates to use as features.

The data frames are "sparse" such that only dates with
data are contained, and thus some of the dates could be
missing, but which must be accounted for.
"""
import datetime as dt

start_date = pd.to_datetime("2012-01-01")
end_date = pd.to_datetime("2015-12-23")

ds = []

# number of days between the end and start date
days = (end_date - start_date).days

for day in range(days):
    ds.append((start_date + dt.timedelta(day)))

date_index = pd.to_datetime(ds)

# create lookup fn for index of date in feature vector
def date_lookup(date):
    return ds.index(date)

# i.e.
# date_lookup(pd.to_datetime('2013-10-14'))

In [43]:
from tqdm import tqdm

def impute_df(df):
    """
    Simple impute of all missing sum of sales per day.
    """
    times = df['ds'].unique()
        
    for d in tqdm(ds):
        if d not in times:
            df.append({
                'time': d,
                'y': 0
            }, ignore_index=True)

In [40]:
# examining one example
df_tengu = demand_for_product_id(products['tengu'])

In [44]:
def days_for_range(start, end):
    return (pd.to_datetime(end) - pd.to_datetime(start)).days

In [46]:
# forecast horizons:
# 1. 24.12.15 to 01.01.16 (9 days)
# 2. weeks of january (30 days (1.1. is already accounted for))
# 3. months of january to march
days = days_for_range("2015-12-24", "2016-03-31")
print(days)

98


In [52]:
def forecast(df):
    p = Prophet(changepoints=None)
    p.fit(df)
    future = p.make_future_dataframe(periods=days + 1)
    forecast = p.predict(future)

    return forecast

In [93]:
def process_product(product):
    df = demand_for_product_id(products[product])
    f = forecast(df)

    days_of_december = f.iloc[-99:-90]
    week1_of_jan = f.iloc[-88:-81]
    week2_of_jan = f.iloc[-81:-74]
    week3_of_jan = f.iloc[-74:-67]
    week4_of_jan = f.iloc[-67:-60]
    days_of_january = f.iloc[-91:-60]
    days_of_february = f.iloc[-60:-31]
    days_of_march = f.iloc[-31:]

    ## values
    # http://kalenderwoche.net/alle-kalenderwochen-2016.php
    # kw1: 4.1.-10.1.
    # kw2: 11.1-17.1.
    # kw3: 18.1.-24.1.
    # kw4: 25.1.-31.1.

    day_labels = list(map(lambda d: str(d),  #).strftime('%Y-%m-%d'),
                           days_of_december['ds'].values))
    days = days_of_december['yhat'].values

    week_labels = ['kw1', 'kw2', 'kw3', 'kw4']
    weeks = [np.sum(week1_of_jan['yhat'].values),
             np.sum(week2_of_jan['yhat'].values),
             np.sum(week3_of_jan['yhat'].values),
             np.sum(week4_of_jan['yhat'].values),]

    month_labels = ['jan', 'feb', 'march']
    months = [np.sum(days_of_january['yhat'].values),
              np.sum(days_of_february['yhat'].values),
              np.sum(days_of_march['yhat'].values)]
    
    labels = (day_labels,
              week_labels,
              month_labels)
    values = (days, weeks, months)

    return (labels, values)

In [94]:
%%time
labels, tengu_forecasts = process_product('tengu')

CPU times: user 6.58 s, sys: 113 ms, total: 6.7 s
Wall time: 6.81 s


In [99]:
final_data = {}
labels = None

for p in tqdm(products):
    l, data = process_product(p)

    if labels is None:
        labels = l

    final_data[p] = data

100%|██████████| 20/20 [10:20<00:00, 48.13s/it]


In [142]:
!rm ../output/aufgabe_1.csv

In [143]:
with open('../output/aufgabe_1.csv', 'a') as f:
    # write header
    cols = ['product']
    
    cols += list(map(lambda d: str(d).split('T')[0],
                           labels[0]))
    cols += labels[1]
    cols += labels[2]

    f.write(','.join(cols) + '\n')

    # write data
    for d in final_data:
        dat = ','.join(list(map(str, np.hstack(final_data[d]))))
        f.write(d + ',' + dat + '\n')
    

In [144]:
!cat ../output/aufgabe_1.csv

product,2015-12-24,2015-12-25,2015-12-26,2015-12-27,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-01,kw1,kw2,kw3,kw4,jan,feb,march
tengu,3.50022570026,3.49657581067,3.57088389134,3.61727040508,3.42534020988,3.36050428085,3.38005273529,3.43187317959,3.42250476185,23.4405107312,22.9641648498,22.7163329443,22.6562613108,102.22638616,95.3057224648,98.3898222454
mackinaw,3.41983750081,3.49514681462,3.64505825055,3.74387073566,3.58171143315,3.54216006465,3.49265510493,3.45320703855,3.52225147165,24.8996731669,24.5876231515,24.4305498438,24.5867666361,109.448921392,101.327899087,103.957626068
ishtar,3.41983750081,3.49514681462,3.64505825055,3.74387073566,3.58171143315,3.54216006465,3.49265510493,3.45320703855,3.52225147165,24.8996731669,24.5876231515,24.4305498438,24.5867666361,109.448921392,101.327899087,103.957626068
hulk,3.25018591799,3.28587613413,3.42627750018,3.51647122044,3.33056912551,3.28847167906,3.28918741189,3.2607718588,3.29549423605,23.4449095592,23.4525868431,23.537036827

In [145]:
!cat ../output/aufgabe_1.csv | wc -l

      21


In [146]:
len(final_data)

20